In [29]:

from gevent.pool import Pool
import dotenv
from app.logging_config import logger
dotenv.load_dotenv(dotenv.find_dotenv(), override=True)
import requests
import time
import json
import os


def format_sources_as_markdown(sources, aggregate_score):
    markdown_string = f"The plagiarism check returned an aggregate score of {aggregate_score}%.\n\n"
    markdown_string += "Here are the sources it was copied from:\n"
    for index, source in enumerate(sources, start=1):
        markdown_string += f"{index}. [{source['title']}]({source['url']})\n"
    return markdown_string

def scan_and_wait_for_results(code, filename="run.txt", timeout=300):
    SCANNER_URL = os.getenv('SCANNER_URL')
    scan_url = f"{SCANNER_URL}/scan/cvS0yn9lbI9Cnwrae7gM"  # Adjust this URL to your actual scan endpoint
    results_url = f"{SCANNER_URL}/results/cvS0yn9lbI9Cnwrae7gM"

    # Data to send to the scan endpoint
    scan_data = {
        'text': code,
        'filename': filename
    }

    # Make the initial request to the scan endpoint
    response = requests.post(scan_url, json=scan_data)
    if response.status_code == 200:
        print("Scan initiated successfully.")
        scan_id = response.json().get('scan_id')
        print("Scan ID:", scan_id)
    else:
        print("Failed to initiate scan:", response.text)
        return None

    # Polling the results endpoint until a valid result is obtained or timeout
    start_time = time.time()
    while time.time() - start_time < timeout:
        result_response = requests.get(results_url + "/" + str(scan_id))
        if result_response.status_code == 200:
            result_data = result_response.json()
            if "results" in result_data:
                print("Results retrieved successfully:")
                final =  json.loads(result_data["results"]["result"])["results"]#['score']
                
                return final
        time.sleep(5)  # wait for 5 seconds before next poll

    print("Failed to retrieve results within the timeout period.")
    return None

def run_check(user_prompt=None, ideal_response=None, checks={}):
    if ideal_response:
        res = scan_and_wait_for_results(ideal_response)
        score = "FAIL" if res["score"]['aggregatedScore'] > checks["aggregatedScore_threshold"] else "PASS"
      
    elif user_prompt:
        res = scan_and_wait_for_results(user_prompt)
        score = "FAIL" if res["score"]['aggregatedScore'] > 30.0 else "PASS"
       
        
        
 #format other part of result
    formatted_output = format_sources_as_markdown(res["internet"], 
                                                  res["score"]['aggregatedScore'])
        
        
    check_result =  {
            "evaluation_result": score,
            "explanation_for_the_evaluation_result": formatted_output,
        }
    
    return check_result



class EvaluationPlagiarismChecker():

    def evaluate(
        self,
        input_data,
        config={"checks": ["plagiarism", "ai_generated"], "aggregatedScore_threshold":30},
        input_validation=True,
        parse=None,
        format_to_issues_scores=False,
    ):

        # happen here
        user_prompt_content = input_data.get("user_prompt", {}).get("content")
        ideal_response_content = input_data.get("alternative_last_ai_reply", {}).get("content")
        result = run_check(user_prompt=user_prompt_content, ideal_response=ideal_response_content, checks=config)
        
        
       

        return {"result":result}



In [30]:
evaluation = {
  "alternative_last_ai_reply": {
  "content": """n = 10
                num1 = 0
                num2 = 1
                next_number = num2  
                count = 1

                while count <= n:
                    print(next_number, end=" ")
                    count += 1
                    num1, num2 = num2, next_number
                    next_number = num1 + num2
                print()""",
    "role": "user"
  },
  "conversation_metadata": {
    "Domain": "Web Development",
    "File System": "No",
    "L1 Taxonomy": "Code editing/rewriting",
    "Programming Language:": "Python",
    "Prompt Structure": "Normal",
    "Subtopic": "Applying Input Sanitization Techniques in Django to Prevent SQL Injection",
    "Task Difficulty": "Easy",
    "Turn Length": "Medium (2-4 turns)",
    "Use Case": "An e-commerce platform implements input sanitization techniques in Django to filter user's input data in order to maintain the security of the website and prevent SQL Injection attacks from malicious users.",
    "batchId": 764,
    "batchName": "Batch 030",
    "projectId": 31,
    "projectName": "[xxxx_COD_PREF] Llama4-Coding"
  },
  "human_review": [
    {},
    {}
  ],
  "last_ai_reply": [
    {
      "content": "You're right to be concerned about edge cases and potential pitfalls. Here are some additional considerations to ensure maximum security and accuracy:\n\n1.  **Input Validation and Sanitization:**\n\n    *   While parameterized queries protect against SQL injection, it's still essential to validate and sanitize user input to prevent other types of attacks, such as cross-site scripting (XSS).\n    *   Consider using Django's built-in `django.utils.html.escape()` function to escape any HTML special characters in the user input.\n    *   You can also use regular expressions to validate the input format and reject any invalid or suspicious input.\n2.  **Wildcard Characters:**\n\n    *   As you mentioned, wildcard characters like `%` and `_` can impact the query's behavior.\n    *   To prevent users from exploiting these characters, you can escape them in the input string.\n    *   For example, you can replace `%` with `\\%` and `_` with `\\_` to treat them as literal characters.\n3.  **Case Sensitivity:**\n\n    *   The `LIKE` operator is case-sensitive, which means it will treat uppercase and lowercase characters as distinct.\n    *   If you want to perform a case-insensitive search, consider using the `ILIKE` operator (if supported by your database) or converting both the input and the database column to lowercase or uppercase.\n4.  **Query Performance:**\n\n    *   Using the `LIKE` operator with a wildcard at the beginning of the pattern (e.g., `%user_input%`) can lead to poor query performance, as the database needs to scan the entire table.\n    *   Consider using full-text search indexes or more advanced search algorithms to improve query performance.\n5.  **Django's ORM:**\n\n    *   As I mentioned earlier, using Django's ORM provides an additional layer of protection against SQL injection and other security risks.\n    *   If possible, consider rewriting your query using the ORM to take advantage of its built-in security features.\n\nHere's an updated version of your code that addresses some of these concerns:\n\n```python\nfrom django.db import connection\nimport re\n\ndef search_products(user_input):\n    # Input validation and sanitization\n    user_input = re.sub(r'[^a-zA-Z0-9\\s]', '', user_input)  # Remove special characters\n    user_input = django.utils.html.escape(user_input)  # Escape HTML special characters\n\n    # Escape wildcard characters\n    user_input = user_input.replace('%', '\\\\%').replace('_', '\\\\_')\n\n    query = \"SELECT * FROM products WHERE name ILIKE %s\"  # Use ILIKE for case-insensitive search\n    params = [f\"%{user_input}%\"]\n\n    with connection.cursor() as cursor:\n        cursor.execute(query, params)\n        results = cursor.fetchall()\n    return results\n```\n\nKeep in mind that this is just an example, and you should adapt it to your specific use case and security requirements.\n\nAs for additional pitfalls with the parameterized approach, here are a few:\n\n*   **Inconsistent parameter formatting:** Make sure to use the same formatting for all parameters in your query. Inconsistent formatting can lead to errors or security vulnerabilities.\n*   **Incorrect parameter types:** Ensure that you're using the correct data type for each parameter. Using the wrong type can cause errors or security issues.\n*   **Overly complex queries:** While parameterized queries provide protection against SQL injection, overly complex queries can still be vulnerable to other types of attacks. Keep your queries simple and focused on the specific task at hand.",
      "role": "assistant"
    },
    {
      "content": "Your updated code is a significant improvement over the original version, as it uses parameterized queries to prevent SQL injection. However, there are still a few edge cases and potential issues to consider:\n\n1.  **Special characters and wildcard symbols**: As you mentioned, inputs containing special characters or wildcard symbols (e.g., `%`, `_`) could impact the query's behavior. In the context of SQL's `LIKE` operator, these characters have special meanings:\n    *   `%` matches any number of characters (including zero).\n    *   `_` matches exactly one character.\n\n    To mitigate this, you can escape these characters in the user input. Django's ORM provides a way to do this automatically, but when using raw SQL queries, you'll need to handle it manually.\n\n    Here's an example of how you can escape these characters:\n\n    ```python\nuser_input = user_input.replace('\\\\', '\\\\\\\\').replace('%', '\\\\%').replace('_', '\\\\_')\nuser_input = f\"%{user_input}%\"\n```\n\n    This code replaces any occurrences of `\\`, `%`, and `_` with their escaped versions.\n\n2.  **Input validation and sanitization**: While parameterized queries prevent SQL injection, it's still essential to validate and sanitize user input to prevent other types of attacks or unexpected behavior.\n\n    Consider adding input validation to ensure that the user input conforms to your expectations. For example, you might want to check that the input is not excessively long or contains only specific characters.\n\n    Django provides various tools for input validation, such as the `django.core.validators` module.\n\n3.  **Case sensitivity**: The `LIKE` operator in SQL is case-sensitive. If you want to perform a case-insensitive search, you can use the `ILIKE` operator instead (if your database supports it).\n\n    Alternatively, you can convert both the user input and the database values to lowercase or uppercase using the `LOWER()` or `UPPER()` functions:\n\n    ```python\nquery = \"SELECT * FROM products WHERE LOWER(name) LIKE LOWER(%s)\"\n```\n\n4.  **Django's ORM**: As I mentioned earlier, Django's ORM provides a more convenient and Pythonic way to interact with your database. If possible, consider using the ORM instead of raw SQL queries.\n\n    The ORM takes care of many security concerns, including SQL injection and input validation. It also provides a more abstract and database-agnostic way to perform queries.\n\nHere's an updated version of your code that incorporates these suggestions:\n\n```python\nfrom django.db import connection\n\ndef search_products(user_input):\n    # Input validation and sanitization\n    if len(user_input) > 100:\n        raise ValueError(\"Input too long\")\n\n    # Escape special characters\n    user_input = user_input.replace('\\\\', '\\\\\\\\').replace('%', '\\\\%').replace('_', '\\\\_')\n    user_input = f\"%{user_input}%\"\n\n    query = \"SELECT * FROM products WHERE LOWER(name) LIKE LOWER(%s)\"\n    with connection.cursor() as cursor:\n        cursor.execute(query, [user_input])\n        results = cursor.fetchall()\n    return results\n```\n\nKeep in mind that this is just an example, and you should adapt it to your specific use case and requirements. Additionally, consider using Django's ORM if possible, as it provides a more secure and convenient way to interact with your database.",
      "role": "assistant"
    }
  ],
  "previous_messages": [
    {
      "content": "I’m working on an e-commerce platform in Django and need to apply input sanitization techniques to prevent SQL injection, especially since users will be submitting product search queries. Here’s a function I implemented to search for products based on user input, but I’m not sure if it’s fully secure:\n\npython\n\nfrom django.db import connection\n\ndef search_products(user_input):\n    query = f\"SELECT * FROM products WHERE name LIKE '%{user_input}%'\"\n    with connection.cursor() as cursor:\n        cursor.execute(query)\n        results = cursor.fetchall()\n    return results\n\nIn this code, I’m using raw SQL with cursor.execute() to run a search query based on the user_input variable. However, I know that directly incorporating user_input could make the platform vulnerable to SQL injection attacks if someone enters malicious SQL code. Could you help me rewrite this code to prevent SQL injection, ensuring that user input is safely sanitized and the platform remains secure?",
      "role": "user"
    },
    {
      "content": "You're right to be concerned about SQL injection. Django provides a way to safely include user input in your SQL queries using parameterized queries. Here's how you can modify your function to prevent SQL injection:\n\n```python\nfrom django.db import connection\n\ndef search_products(user_input):\n    query = \"SELECT * FROM products WHERE name LIKE %s\"\n    params = [f\"%{user_input}%\"]\n    \n    with connection.cursor() as cursor:\n        cursor.execute(query, params)\n        results = cursor.fetchall()\n    return results\n```\n\nIn this revised code:\n\n*   We define the query with a placeholder (`%s`) for the user input. This placeholder will be replaced with the actual input by the database driver.\n*   We create a separate list (`params`) to hold the user input. This list is passed as the second argument to `cursor.execute()`.\n*   The database driver will safely escape the user input and replace the placeholder in the query. This prevents any malicious SQL from being executed.\n\nHowever, it's worth noting that using raw SQL queries in Django is generally discouraged. Django provides a powerful ORM (Object-Relational Mapping) system that allows you to interact with your database in a safer and more Pythonic way.\n\nHere's how you can rewrite the function using Django's ORM:\n\n```python\nfrom django.db.models import Q\nfrom .models import Product  # Assuming you have a Product model\n\ndef search_products(user_input):\n    results = Product.objects.filter(Q(name__icontains=user_input))\n    return results\n```\n\nIn this version:\n\n*   We use the `Q` object from Django's ORM to build a query that filters products based on the `name` field.\n*   The `icontains` lookup is used to match products with names that contain the user input (case-insensitive).\n*   The ORM takes care of sanitizing the user input and preventing SQL injection.\n\nUsing Django's ORM is generally the recommended approach, as it provides a higher level of abstraction and safety. However, if you need to use raw SQL queries for specific reasons, make sure to use parameterized queries to prevent SQL injection.",
      "role": "assistant"
    },
    {
      "content": "I implemented the recommended changes to protect against SQL injection by using parameterized queries, but I’m still unsure if I’ve missed any additional security risks that could arise from processing user-provided input this way. Here’s my updated code:\n\npython\n\nfrom django.db import connection\n\ndef search_products(user_input):\n    query = \"SELECT * FROM products WHERE name LIKE %s\"\n    user_input = f\"%{user_input}%\"\n    with connection.cursor() as cursor:\n        cursor.execute(query, [user_input])\n        results = cursor.fetchall()\n    return results\n\nWhile this approach should theoretically prevent SQL injection by using placeholders, I'm concerned about edge cases, such as inputs containing special characters or wildcard symbols that could impact the query's behavior. Are there any further precautions or Django-specific functions I should apply to ensure maximum security and accuracy of the search functionality? Also, could you point out if there are any additional pitfalls with this parameterized approach that I should be aware of?\n",
      "role": "user"
    }
  ],
  "user_prompt": {
    "content": "I implemented the recommended changes to protect against SQL injection by using parameterized queries, but I’m still unsure if I’ve missed any additional security risks that could arise from processing user-provided input this way. Here’s my updated code:\n\npython\n\nfrom django.db import connection\n\ndef search_products(user_input):\n    query = \"SELECT * FROM products WHERE name LIKE %s\"\n    user_input = f\"%{user_input}%\"\n    with connection.cursor() as cursor:\n        cursor.execute(query, [user_input])\n        results = cursor.fetchall()\n    return results\n\nWhile this approach should theoretically prevent SQL injection by using placeholders, I'm concerned about edge cases, such as inputs containing special characters or wildcard symbols that could impact the query's behavior. Are there any further precautions or Django-specific functions I should apply to ensure maximum security and accuracy of the search functionality? Also, could you point out if there are any additional pitfalls with this parameterized approach that I should be aware of?\n",
    "role": "user"
  }
}

In [ ]:
eval = EvaluationPlagiarismChecker()
rx = eval.evaluate(input_data=evaluation)

In [32]:
rx

{'result': {'evaluation_result': 'FAIL',
  'explanation_for_the_evaluation_result': 'The plagiarism check returned an aggregate score of 100.0%.\n\nHere are the sources it was copied from:\n1. [Module 1-Python prog.-BPLCK105B by Dr.SV.pdf](https://www.slideshare.net/slideshow/module-1-python-prog-bplck105b-by-dr-sv-pdf/272555633)\n2. [Fibonacci series number in C – Complete guide with example](https://codedamn.com/news/c/fibonacci-series)\n3. [Python Program to Print the Fibonacci sequence - GeeksforGeeks](https://www.geeksforgeeks.org/python-program-to-print-the-fibonacci-sequence/)\n4. [Py – #B029 – Netbyzz](https://netbyzz.com/py-b029/)\n5. [Latest 50+ Python Interview Questions and Answers (2024)](https://intellipaat.com/blog/interview-question/python-interview-questions/)\n'}}